In [53]:
import pandas as pd
import re
import string
import sklearn
import csv
import nltk
from nltk.corpus import wordnet 

In [54]:
dataset_file="dataset7(dark knight).csv"
synopsis_file='synopsis_darkknight.txt'
df = pd.read_csv(dataset_file)
file = open(synopsis_file,"r")
text=file.read()
sentences = text.split('. ')
   
df2 = pd.DataFrame(sentences, columns=["synopsis"])
df2.to_csv('synopsis3.csv', index=False)
# print(df2.head())
df_pos=df.loc[df['is_spoiler'] == True]
df_neg=df.loc[df['is_spoiler'] == False]
pos_size=min(1000,df_pos.shape[0])
neg_size=min(1500,df_neg.shape[0])
print(pos_size,neg_size)
train_size=0.8*(pos_size+neg_size)
test_size=0.2*(pos_size+neg_size)
df_train_pos=df_pos.loc[:0.8*pos_size,:]
df_test_pos=df_pos.loc[0.8*pos_size+1:,:]
df_train_neg=df_neg.loc[:0.8*neg_size,:]
df_test_neg=df_neg.loc[0.8*neg_size+1:,:]
# df_train_neg.to_csv('train.csv', index=False)
frames_train=[df_train_pos,df_train_neg]
frames_test=[df_test_pos,df_test_neg]
df_train=pd.concat(frames_train)
df_test=pd.concat(frames_test)
df_train.to_csv('train.csv', index=False)
df_test.to_csv('test.csv', index=False)


1000 1500


In [55]:
#print(df.head())

In [56]:
total_size=len(df.index)

In [57]:
index=[]
for i in range(len(df_train.index)):
    index.append("Comment "+str(i))
#df.insert(loc=0,column='index',value=index)
df_train['index']=index
df_train.set_index('index', inplace=True)

index2=[]
for i in range(len(df2.index)):
    index2.append("Comment "+str(i))
#df.insert(loc=0,column='index',value=index)
df2['index']=index2
df2.set_index('index', inplace=True)


In [58]:
#print(index)

In [59]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
round1 = lambda x: clean_text_round1(x)

In [60]:
data_clean = pd.DataFrame(df_train.review_text.apply(round1))
data_clean2 = pd.DataFrame(df2.synopsis.apply(round1))
# data_clean

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.review_text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index


cv2 = CountVectorizer(stop_words='english')
data_cv2 = cv2.fit_transform(data_clean2.synopsis)
data_dtm2 = pd.DataFrame(data_cv2.toarray(), columns=cv2.get_feature_names())
data_dtm2.index = data_clean2.index
print(data_dtm2.shape)
print(data_dtm.shape)


(248, 1357)
(801, 16868)


In [62]:
import numpy as np
from gensim import matutils, models
import scipy.sparse

In [63]:

id_word_dict={}
for i in range(len(data_dtm.columns)):
    id_word_dict[i]=data_dtm.columns[i]
#print(id_word_dict)
tdm = data_dtm.transpose()

id_word_dict2={}
for i in range(len(data_dtm2.columns)):
    id_word_dict2[i]=data_dtm2.columns[i]
# print(id_word_dict2)
tdm2 = data_dtm2.transpose()

In [64]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

sparse_counts2 = scipy.sparse.csr_matrix(tdm2)
corpus2 = matutils.Sparse2Corpus(sparse_counts2)

In [65]:
def test_model(word_weight,word_weight_synopsis,threshold,threshold2):
    df3=pd.read_csv("test.csv")
    total_size=df3.shape[0]
    print(total_size)
    
    output_list=[]
    counter=0
    for j in range(total_size):
       # print(total_size-j)
        comment_t=df3['review_text'][j]
        if df3['is_spoiler'][j] == 0:
            counter+=1

        comment_t=clean_text_round1(comment_t)
        s1=comment_t.split(" ")
        comment_ts=set(s1)
        comment_ts
        sum=0
        sum2=0
        sum3=0
        for i in word_weight_synopsis.values():
            sum3+=float(i)*float(i)  

        for wor in comment_ts:
            temp_max=0
            synonyms = []
            synonyms.append(wor);
            for syn in wordnet.synsets(wor): 
                for l in syn.lemmas(): 
                    synonyms.append(l.name()) 
            flag=False
            found_word="";
            for i in synonyms: 
                if i in word_weight :
                    found_word=i
                    flag=True                
                if i in word_weight_synopsis and i in word_weight and float(word_weight_synopsis[i])>threshold2:
                    temp_max=max(temp_max,float(word_weight_synopsis[i])*float(word_weight[i]))
                    found_word=i
                    break

            if flag==True:
                sum2+=float(word_weight[found_word])*float(word_weight[found_word])
                sum+=temp_max

        if sum2!=0 :
            #print(sum,sum2,sum3)
            output_list.append(sum/(np.sqrt(sum2)*np.sqrt(sum3)))
        else:
            output_list.append(0)
            
            
    df3['output']=output_list
    cols = list(df3.columns.values)
    df3 = df3[['Unnamed: 0', 'is_spoiler',  'output' ,'review_text']]
    df3.to_csv('final.csv')
    total=0
    false_positive=0
    false_negative=0
    true_positive=0
    true_negative=0

    for i in range(total_size):
        #print(df3['output'][i],threshold,df3['Y'][i])
        if df3['output'][i]>threshold and df3['is_spoiler'][i]==True  :
            true_positive+=1
        if  df3['output'][i]<threshold and df3['is_spoiler'][i]==False :
            true_negative+=1
        if df3['output'][i]>threshold and df3['is_spoiler'][i]==False:
            false_positive+=1
        if df3['output'][i]<threshold and df3['is_spoiler'][i]==True:
            false_negative+=1
    
    accuracy=(true_negative+true_positive)/total_size
    precision=true_positive/(true_positive+false_positive)
    recall=true_positive/(true_positive+false_negative)
    f1score=(2*precision*recall)/(precision+recall)
    return (true_positive,true_negative,false_positive,false_negative,f1score,accuracy)
        

In [66]:
def solve(total_topics,corpus,id_word_dict,corpus2,id_word_dict2):
    lda = models.LdaMulticore(corpus=corpus, id2word=id_word_dict, num_topics=total_topics, passes=1)
    lda2 = models.LdaMulticore(corpus=corpus2, id2word=id_word_dict2, num_topics=total_topics, passes=1)
    
    #print(lda.print_topics())
# word weight dictionary comments
    word_weight={}    
    for i in range(total_topics):
        word_topic=lda.print_topic(i).split('+')
        j=0
        for word_probab in word_topic:
            val,wor=word_probab.split('*')
            wor=str(wor)
            y=len(wor)
            if j==len(word_topic)-1:
                wor = wor[1:y-1]
            else:
                wor = wor[1:y-2]
            if wor in word_weight:
                if word_weight[wor]<val:
                    word_weight[wor]=val
            else :
                 word_weight[wor]=val
            j=j+1
            
# word weight dictionary synopsis
    word_weight_synopsis={}
    for i in range(total_topics):
        word_topic2=lda2.print_topic(i).split('+')
        j=0
        for word_probab in word_topic2:
            val,wor=word_probab.split('*')        
            wor=str(wor)
            #print(wor)
            y=len(wor)
            if j==len(word_topic2)-1:
                wor = wor[1:y-1]
            else:
                wor = wor[1:y-2]
            #print(wor)
            if wor in word_weight_synopsis:
                if word_weight_synopsis[wor]<val:
                    word_weight_synopsis[wor]=val
            else :
                 word_weight_synopsis[wor]=val

            j=j+1
            
    # evaluation
    return (word_weight,word_weight_synopsis)
       
    
    

In [68]:
total_topics=100
passes=50
(word_weight,word_weight_synopsis)=solve(total_topics,corpus,id_word_dict,corpus2,id_word_dict2)


In [ ]:
threshold=0.11
threshold2=0.02
#print(word_weight_synopsis)
(true_positive,true_negative,false_positive,false_negative,f1score,accuracy)=test_model(word_weight,word_weight_synopsis,threshold,threshold2)
print(true_positive,true_negative,false_positive,false_negative,f1score,accuracy)

4044


In [ ]:
print(true_positive,true_negative,false_positive,false_negative,f1score,accuracy)